# Interpolate DUACS gesostrophic vel to times/positions

* read drifter files to get list of U_g, V_g for each
* load in the DUACS file already downloaded using motu client (see aviso_plots.ipynb)
* do the interpolation
* maybe to this as a function 

This notebook works pretty well.

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime
import requests
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
import functions  # requires functions.py from this directory

In [2]:
dr_num=[4444094, 4443240, 4443183, 4442710, 4441770, 4441766, 4441765, 4441338, 4439046, 4435822, 4434552, 4433868, 4431967, 4431769, 4427807, 4427782]
     
num_dr=np.shape(dr_num)

In [3]:
dr_list = []
fname_list = []
fout_list = []
for num in dr_num:
    dr_list.append(str(num))
    fname_list.append('http://smode.whoi.edu:8080/thredds/dodsC/IOP1_2022/drifters/SMODE_PFC_surface_drifter_0-'+str(num)+'.nc')
    fout_list.append('../data/processed/drifters/SMODE_surface_drifter_with_DUACS_0-'+str(num)+'.nc')

In [4]:
# Load DUACS altimetry product from a file that was already extracted using a shell script and the motu client python tool
# A shell script using the motuclient, https://help.marine.copernicus.eu/en/articles/4796533-what-are-the-motu-client-motuclient-and-python-requirements
# Shell script from Ben Greenwood (email 9/21/2022)
'''
echo "$(date -u) download_aviso.sh" >> ./aviso_download.log

start="2022-09-15 00:00:00"
end="$(date -u "+%Y-%m-%d %H:%M:%S")"
out_dir='../'

# download data
motuclient --motu https://nrt.cmems-du.eu/motu-web/Motu --service-id SEALEVEL_GLO_PHY_L4_NRT_OBSERVATIONS_008_046-TDS --product-id dataset-duacs-nrt-global-merged-allsat-phy-l4 --longitude-min -140 --longitude-max -120 --latitude-min 34 --latitude-max 43 --date-min "$start" --date-max "$end" --variable adt --variable err_ugosa --variable err_vgosa --variable sla --variable ugos --variable ugosa --variable vgos --variable vgosa --out-dir "$out_dir" --out-name aviso.nc --user ***** --pwd *****
'''

if 1==2:
    # run the shell script
    ! bash ~/download_aviso.sh
elif:
    file = '../data/external/aviso.nc'
    duacs = xr.open_dataset(file)

In [13]:
#to suppress a FutureWarning
%%capture --no-stderr 

for n in range(len(dr_num)):
    fname=fname_list[n]
    fout = fout_list[n]
    ds = xr.open_dataset(fname)
    new_lon = ds.longitude
    new_lat = ds.latitude
    new_time = ds.time
    ugos = duacs.ugos.interp(time=new_time,latitude=new_lat, longitude=new_lon, method="linear")
    ugosa = duacs.ugosa.interp(time=new_time,latitude=new_lat, longitude=new_lon, method="linear")
    vgos = duacs.vgos.interp(time=new_time,latitude=new_lat, longitude=new_lon, method="linear")
    vgosa = duacs.vgosa.interp(time=new_time,latitude=new_lat, longitude=new_lon, method="linear")
    dsi = xr.Dataset({'ugos': ugos, 'ugosa': ugosa, 'vgos': vgos, 'vgosa': vgosa})
    dsi.to_netcdf(fout)

UsageError: Line magic function `%%capture` not found.


In [ ]:
dsi

In [ ]:
plt.figure()
plt.plot(dsfoo.time, dsfoo.ugos)
fig=plt.gcf()
fig.autofmt_xdate()

In [ ]:
plt.figure()
plt.plot(dsfoo.longitude, dsfoo.latitude,'o')
